# Import

In [1]:
# math and dataframes
import pandas as pd
import numpy as np

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier


# Pipeline and Evaluation
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.tree import plot_tree

# jupyter notebook full-width display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# pandas formatting
pd.set_option('display.float_format', '{:.3f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

# plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
import time
import seaborn as sns
sns.set_theme()

In [2]:
df_10M = pd.read_pickle('df_10M_clustered.pickle')
X_all = pd.read_pickle('X_clustered.pickle')

# Create Datasets for Prediction

In [3]:
y_column = 'in_B100'
X_columns = [
    'mode', 'acousticness', 'danceability', 'duration_ms', 'energy',
    'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence'
]
genre_columns = [
    'is_Adult_Standard', 'is_Rock', 'is_R&B', 'is_Country', 'is_Pop',
    'is_Rap', 'is_Alternative', 'is_EDM', 'is_Metal'
]
cluster_columns = ['cluster', 'cluster2']
other_columns = ['key', 'time_signature', 'genre', 'release_date']


In [4]:
# create a dict with all 'name': (y, X) key match pairs
clusters = {}

# entire predictive dataset
clusters['All'] = (X_all[y_column], X_all[X_columns])

# add genres
for genre in genre_columns:
    title = genre[3:]
    clusters[title] = (X_all[X_all[genre]][y_column], X_all[X_all[genre]][X_columns])
    
# add clusters
for n in sorted(X_all['cluster'].unique()):
    title = genre[3:]
    clusters['cluster1_' + str(n)] = (X_all[X_all['cluster'] == n][y_column], X_all[X_all['cluster'] == n][X_columns])
    
for n in sorted(X_all['cluster2'].unique()):
    title = genre[3:]
    clusters['cluster2_' + str(n)] = (X_all[X_all['cluster2'] == n][y_column], X_all[X_all['cluster2'] == n][X_columns])

In [5]:
# clusters['All'][1].head()

In [6]:
# list(clusters.keys())

# Make Predictions Using Default ML Settings
### COMPARE ALGORITHMS

In [7]:
ML_algorithms = [
    LogisticRegression,
    DecisionTreeClassifier,
#     SVC,  # too slow, would take days/months
    RandomForestClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier
]

In [8]:
# these are slow, even the simlest models
# let's see how long they take with a small and medium dataset
# then we can estimate how long each algorithm will take to run

small = X_all.sample(10_000)
y_small = small[y_column]
X_small = small[X_columns]

medium = X_all.sample(100_000)
y_medium = medium[y_column]
X_medium = medium[X_columns]

large = X_all.sample(1_000_000)
y_large = large[y_column]
X_large = large[X_columns]

In [9]:
def ml_time(X, y, ml_n):
    """testing how long each algorithm takes"""
    grid = GridSearchCV(
        estimator=ML_algorithms[ml_n](),
        param_grid={},
        cv=5
    )
    %time grid.fit(X, y)

In [10]:
# small datasets
for i in range(len(ML_algorithms)):
    print(str(ML_algorithms[i])[:-2].split('.')[-1], end=': ')
    ml_time(X_small, y_small, i)

LogisticRegression: Wall time: 131 ms
DecisionTreeClassifier: Wall time: 228 ms
RandomForestClassifier: Wall time: 3.69 s
AdaBoostClassifier: Wall time: 2.33 s
GradientBoostingClassifier: Wall time: 9.35 s


In [11]:
# medium datasets
for i in range(len(ML_algorithms)):
    print(str(ML_algorithms[i])[:-2].split('.')[-1], end=': ')
    ml_time(X_medium, y_medium, i)

LogisticRegression: Wall time: 981 ms
DecisionTreeClassifier: Wall time: 3.74 s
RandomForestClassifier: Wall time: 1min 2s
AdaBoostClassifier: Wall time: 21.4 s
GradientBoostingClassifier: Wall time: 1min 38s


In [12]:
# medium datasets
for i in range(len(ML_algorithms)):
    print(str(ML_algorithms[i])[:-2].split('.')[-1], end=': ')
    ml_time(X_large, y_large, i)

LogisticRegression: Wall time: 11.8 s
DecisionTreeClassifier: Wall time: 1min 3s
RandomForestClassifier: Wall time: 25min 24s
AdaBoostClassifier: Wall time: 6min 8s
GradientBoostingClassifier: Wall time: 21min 36s


Time Complexity Notes:
* Naive Bayes: O(n*d)
* Logistic Regression: O(n*d)
* Decision Tree: O(n*log(n))
* SVC: O(n^2)
    * Note from medium.com: "if n is large, avoid using SVM."
* Random Forest: O(n*log(n)*k)
    * i assume other boosted trees are similar
    
    
REFERENCE:
https://medium.com/analytics-vidhya/time-complexity-of-ml-models-4ec39fad2770

* based on the above times, it will take about this long to calculate without resampling:
    * LogisticRegression: 1 minute
    * DecisionTreeClassifier: Wall time: 15 minutes
    * RandomForestClassifier: Wall time: 7 hours
    * AdaBoostClassifier: Wall time: 58 minutes
    * GradientBoostingClassifier: Wall time: 4 hours

In [ ]:
%%time
# test logistic regression

grid = GridSearchCV(
    estimator=ML_algorithms[0](),
    param_grid={},
    cv=5
)
grid.fit(X, y)

grid.cv_results_

In [ ]:
# almost 100% accurate by predicting no songs are hits
# need to deal with unbalanced data, but for now, it looks like it's working
1 - sum(y) / y.shape[0]

In [ ]:
%%time
# check decision tree, different method, just the score
cross_val_score(ML_algorithms[1](), X, y, cv=5, scoring='accuracy').mean()